In [ ]:
# Task 2: Web scraping & data preparation - STU59362
!pip install selenium

# Import libraries - STU59362
import selenium
from selenium import webdriver
import pandas as pd

# Driver setup for opening URl in Google Chrome 
!pip install webdriver-manager

# Import Driver
from selenium import webdriver
PATH = 'chromedriver.exe'
browser = webdriver.Chrome(executable_path = PATH)

# Specify Search URL - STU59362
URL = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans'
browser.get(URL)

# STU59362 = To generate a blank data frame to store usernames, dates and postings of 100 users.
Forum_Discussions = pd.DataFrame(columns = ['User_Name', 'Date','Forum_Discussions'])

y = 1
while (y<=435): # Total number of web pages to be crawled is 435
    # Running while loop only till we get 500 comments while scrolling to the bottom of the website
    if (len(Forum_Discussions)<500):
        url = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p' + str(y)
        browser.get(url)
        # Finding elements
        usernames = browser.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
        message_usernames = []
        for x in usernames:
            message_usernames.append(x.get_attribute('id'))

        for j in message_usernames:
            
            #Fetching the user names of each user on a page
            username_element = browser.find_elements_by_xpath('//*[@id="' + j +'"]/div/div[2]/div[1]/span[1]/a[2]')[0]
            username = username_element.text
                        
            #Fetching the date of each user on a page
            posting_date = browser.find_elements_by_xpath('//*[@id="' + j +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
            date = posting_date.get_attribute('title')
          
            #Fetching the forum postings of each user on a page
            user_post = browser.find_elements_by_xpath('//*[@id="' + j +'"]/div/div[3]/div/div[1]')[0]
            message = user_post.text
            
            # To extract any present Block Quote
            block_quote = browser.find_element_by_xpath('//*[@id="' + j + '"]/div/div[3]/div/div[1]')
            block_quote_class = block_quote.find_elements_by_class_name('UserQuote')
            block_text = ''
            if len(block_quote_class)>0:
                block_text = block_quote_class[0].text
            
            # To replace block quotes
            message = message.replace(block_text,"")
            
           #Adding userid, date, and comment for each user posting in the dataframe 
        
            Forum_Discussions.loc[len(Forum_Discussions)] = [username,date,message]
            #print(username,date,message)
        y=y+1
 
    else:
        break
print(Forum_Discussions)

print (Forum_Discussions.head())


# import necessary libraries
from time import sleep
from string import punctuation
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import copy
import matplotlib.pyplot as plt
%pylab inline
import numpy as np
import pandas as pd


# Making a copy of the original dataframe and converting the data into a standardised csv file - STU59362
import copy
Forum_Discussions_copy = copy.deepcopy(Forum_Discussions)

def remove_space(s):
    return s.replace("\n"," ")

Forum_Discussions_copy['Forum_Discussions'] = Forum_Discussions_copy['Forum_Discussions'].apply(remove_space)
Forum_Discussions_copy.to_csv('postings.csv', header=True, sep=',') 


# Cleaning the data
# A "veh_models" csv (UTF-8 encoded) file had been created and saved in the working directory.
# It contains the brands and models of vehicles 

Forum_Discussions_copy = pd.read_csv('postings.csv') #reading postings
#Reading vehicle models
veh_models = pd.read_csv("veh_models.csv", header = None, names = ['brand','model'], encoding='windows-1252')
Forum_Discussions_copy = Forum_Discussions_copy.dropna()
Forum_Discussions_copy.reset_index(inplace  = True)

# Cleaning process "functions"

def removepunc(item):
    for p in punctuation:
        item = item.lstrip().replace(p,'')
    return item

def lowerize(x):
    return x.lower()

Forum_Discussions_copy['Forum_Discussions_clean'] = Forum_Discussions_copy['Forum_Discussions'].apply(removepunc).apply(lowerize)
veh_models['brand'] = veh_models['brand'].apply(removepunc)



def vehmodel_to_brand(s):
    for k in veh_models.index.values:
        s = s.replace(veh_models["model"][k].lower(),veh_models["brand"][k].lower())
    return s
Forum_Discussions_copy['Forum_Discussions_vehmodel_replace'] = Forum_Discussions_copy['Forum_Discussions_clean'].apply(vehmodel_to_brand)


Forum_Discussions_copy['Forum_Discussions_appear'] = Forum_Discussions_copy['Forum_Discussions_vehmodel_replace'].apply(word_tokenize).apply(set).apply(list)

# Remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 
    
Forum_Discussions_copy['final_Forum_Discussions'] =  Forum_Discussions_copy['Forum_Discussions_appear'].apply(remove_stopwords)
# filtered_sentence = [w for w in temp['Forum_Discussions_appear'] if not w in stop_words]


count = []
for j in range(len(Forum_Discussions_copy)):
    count+=Forum_Discussions_copy['final_Forum_Discussions'][j]
    

from nltk import FreqDist
freq_dist = nltk.FreqDist(count)



unique_veh_models = veh_models['brand'].drop_duplicates().tolist()


# Frequency Distribution of 600 leading brands
leading_words = freq_dist.most_common(600)
leading_brands = []
for (key, items) in leading_words:
    if key in unique_veh_models:
        model_sum_total = (key,items)
        leading_brands.append(model_sum_total)
        
        
# Leading 20 vehicle brands
leading_20_brands_sum_total = leading_brands[:20]
print ('The leading 20 vehicle brands together with their frequency distributions are:\n' , leading_20_brands_sum_total[:20])




print (veh_models)

#Calculating lift ratios for association between 10 leading brands
# Fetching the top 10 brands.
leading_10_brands =[]
for brand, count in leading_10_brands_counts:
    leading_10_brands.append(brand)

new_df = pd.DataFrame(columns = leading_10_brands)

def brand_mentioned(item):
    if brand in item:
        return 1
    else:
        return 0
      
for brand in leading_10_brands:
    new_df[brand] = Forum_Discussions_copy['final_Forum_Discussions'].apply(brand_mentioned)
    
# Calculating lift among top brands
df2=pd.DataFrame(columns = leading_10_brands)
for i in range(len(leading_10_brands)):
    new_list = []
    for j in range(len(leading_10_brands)):
        if (i!=j):
            numerator = ((new_df[leading_10_brands[i]] + new_df[leading_10_brands[j]]) > 1).sum()
            denominator = new_df[leading_10_brands[j]].sum()*new_df[leading_10_brands[i]].sum()
            lift = numerator*len(new_df)/denominator
            df2.loc[leading_10_brands[i],leading_10_brands[j]] = lift
print ('Below are the lift ratios among leading brands\n')
df2

# Leading 10 vehicle brands
leading_10_brands_sum_total = leading_brands[:10]
print ('The leading 10 vehicle brands together with their frequency distributions are:\n' , leading_10_brands_sum_total[:10])


# Calculating lift ratios for association between 10 leading brands

# Fetching the leading 10 brands.

leading_10_brands =[]
for brand, count in leading_10_brands_sum_total:
    leading_10_brands.append(brand)

new_df = pd.DataFrame(columns = leading_10_brands)

def brand_mentioned(item):
    if brand in item:
        return 1
    else:
        return 0
      
for brand in leading_10_brands:
    new_df[brand] = Forum_Discussions_copy['final_Forum_Discussions'].apply(brand_mentioned)
    
# Lift ratio calculation among leading brands

df2=pd.DataFrame(columns = leading_10_brands)
for x in range(len(leading_10_brands)):
    new_list = []
    for y in range(len(leading_10_brands)):
        if (x!=y):
            numerator = ((new_df[leading_10_brands[x]] + new_df[leading_10_brands[y]]) > 1).sum()
            denominator = new_df[leading_10_brands[y]].sum()*new_df[leading_10_brands[x]].sum()
            lift = numerator*len(new_df)/denominator
            df2.loc[leading_10_brands[x],leading_10_brands[y]] = lift
print ('The lift ratios among leading brands are:\n')
df2

# Calculating the dissimilarity matrix which is the input for plotting MDS plot
dissimilarity_matrix = 1/df2
np.fill_diagonal(dissimilarity_matrix.values, 0)

# Plotting MDS plot
from sklearn import manifold
seed = np.random.RandomState(seed=3)
mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, 
      random_state=seed, dissimilarity="precomputed", n_jobs=1)
results = mds.fit(dissimilarity_matrix)
coords = results.embedding_

plt.subplots_adjust(bottom = 0.1)
plt.scatter(
    coords[:, 0], coords[:, 1], marker = 'o'
    )
for label, x, y in zip(leading_10_brands, coords[:, 0], coords[:, 1]):
    
    plt.annotate(
        label,
        xy = (x, y), xytext = (-30, 30),
        textcoords = 'offset points', ha = 'right', va = 'bottom',
        bbox = dict(boxstyle = 'round,pad=0.5', fc = 'blue', alpha = 0.5),
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))

plt.show()

print(df2)

#Checking for null values
print(Forum_Discussions.isnull().sum())

# Most frequent features of cars mentioned in the forum discussions

features = pd.read_csv('features.csv') # Read the contents of the file

def word_to_features(z):
    z = " ".join(str(y) for y in z)
    for k in features.index.values:
        z = z.replace(features["Feature"][k].lower(),features["Mapping"][k].lower())
    return z
Forum_Discussions_copy['Forum_Discussions_features_replace'] = Forum_Discussions_copy['final_Forum_Discussions'].apply(word_to_features)


count = []
for k in range(len(Forum_Discussions_copy)):
    count+=Forum_Discussions_copy['Forum_Discussions_features_replace'][k]
feat_freq = nltk.FreqDist(count)
features_unique = features['Mapping'].drop_duplicates().tolist() # The unique mapping features

leading_words = freq_dist.most_common(2000) # Frequecy distibution of the most 2000 common words
leading_features = []
for (key, items) in leading_words:
    if key in features_unique:
        feature_counts = (key,items)
        leading_features.append(feature_counts) 
        

leading_5_features_counts = leading_features[:5]

# The dataframe 
print('The shape of the data is \n', features, '\n')

# The unique mapping features
print('The unique mapping features are:\n', features_unique, '\n')

print ('The frequency distribution of the leading 5 features are:\n' , leading_5_features_counts)

# Most strongly associated of the features among the leading 5 brands

# Fetching leading 5 features
leading_5_features =[]
for feature, count in leading_5_features_counts:
    leading_5_features.append(feature)

features_df = pd.DataFrame(columns = leading_5_features)

def feature_mentioned(item):
    if feature in item:
        return 1
    else:
        return 0
      
for feature in leading_5_features:
      
    features_df[feature] = Forum_Discussions_copy['Forum_Discussions_features_replace'].apply(feature_mentioned)    


# Calculating Lift between leading 5 brands and leading 5 features
df3=pd.DataFrame(columns = leading_5_features)
leading_5_brands = leading_10_brands[:5]
for i in range(len(leading_5_brands)):
    new_list = []
    for j in range(len(leading_5_features)):
        numerator = ((new_df[leading_5_brands[i]] + features_df[leading_5_features[j]]) > 1).sum()
        denominator = new_df[leading_5_brands[i]].sum()*features_df[leading_5_features[j]].sum()
        lift_brand_features = numerator*len(features_df)/denominator
        df3.loc[leading_5_brands[i],leading_5_features[j]] = lift_brand_features

print ('The lift ratios between leading 5 brands and leading 5 features are: \n\n', df3)

print('\n', end="")

#Display the line graph chart of the lift ration
df3.plot()
plt.show()


print ('The lift ratios between leading 5 brands and leading 5 features are: \n')
df3


# The most desired target car to own or likely to purchse

desires = pd.read_csv("desires.csv") # Read the dataframe

# Common target ambitions mentioned in the forum discussions replaced by "desires"

def ambition(s):
    #s = " ".join(str(x) for x in s)
    for i in desires['word'].index.values:
        s = s.replace(desires['word'][i],desires['target'][i])
    return s
Forum_Discussions_copy['Forum_Discussions_amb_replace'] = Forum_Discussions_copy['Forum_Discussions_features_replace'].apply(ambition)


ambition_df = pd.DataFrame(columns = ['desires'])

def ambition_mentioned(item):
    if amb in item:
        return 1
    else:
        return 0

for amb in desires['target'].unique():
    ambition_df[amb] = Forum_Discussions_copy['Forum_Discussions_amb_replace'].apply(ambition_mentioned)

    
    # Calculating Lift ratios between leading 5 brands and desires
    ambition_df2=pd.DataFrame(columns = ['desires'])
leading_5_brands = leading_10_brands[:5]
for i in range(len(leading_5_brands)):
    new_list = []
    for j in range(len(desires['target'].unique())):
        numerator = ((new_df[leading_5_brands[i]] + ambition_df['desires']) > 1).sum()
        denominator = new_df[leading_5_brands[i]].sum()*ambition_df['desires'].sum()
        lift_brand_target = numerator*len(ambition_df)/denominator
        ambition_df2.loc[leading_5_brands[i],'desires'] = lift_brand_target

print ('The lift ratios between leading 5 brands and desires are: \n\n', ambition_df2)

print('\n', end="")

#Display the line graph chart of the lift ratio

ambition_df2.plot()
plt.show()

ambition_df2

desires.head()

print(features)

# The total number of common words in desire.csv file
desires.select_dtypes('object').nunique()

# The actual words changed to desires
print (desires['word'].unique())

# The "desires.csv" file
print (desires)

desires['word'].value_counts()

# Exporting the final cleaned data frame

final_df = Forum_Discussions_copy[['User_Name', 'Date','Forum_Discussions','Forum_Discussions_amb_replace']]
final_df.columns = ['User_Name', 'Date','Forum_Discussions','cleaned_Forum_Discussions']
final_df.to_csv('final_data_file.csv', header=True, sep=',') 

# Reading the "final_data_file"

final_data_file = pd.read_csv('final_data_file.csv')

#Displaying the final cleaned data file
print(final_data_file)

# Displaying final data file
final_data_file

#To view the full content of the cells of a dataframe
pd.set_option('max_colwidth', None)

print(final_data_file)

print(final_data_file.cleaned_Forum_Discussions)

print(Forum_Discussions)

print (final_df)

final_data_file

